# Satellite Intelligence for Catastrophic Natural Disaster Recovery: Assessing Damage and First Responder Priorities through Geospatial Imagery

## Jeremiah Fa'atiliga, Ravita Kartawinata, Sowmiya Kanmani Maruthavanan

# Importing Libraries

In [1]:
import os
import json
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")


The xBD dataset provides annotated high-resolution satellite imagery for assessing building damage, consisting of JSON files and image files. This project focuses on analyzing pre- and post-disaster imagery related to hurricanes.

# Define Data

In [2]:
# Read JSON files
def create_dataframes(directory):
    pre_data = []
    post_data = []

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            content = json.load(file)

        # Filter by hurricane natural disaster only
        disaster = content['metadata'].get('disaster')
        if disaster and "hurricane" in disaster.lower():
            img_name = content['metadata'].get('img_name', "")
            data = {
                'img_name': img_name,
                'xy': content['features'].get('lng_lat'),
                'sensor': content['metadata'].get('sensor'),
                'provider_asset_type': content['metadata'].get('provider_asset_type'),
                'gsd': content['metadata'].get('gsd'),
                'capture_date': content['metadata'].get('capture_date'),
                'off_nadir_angle': content['metadata'].get('off_nadir_angle'),
                'pan_resolution': content['metadata'].get('pan_resolution'),
                'sun_azimuth': content['metadata'].get('sun_azimuth'),
                'sun_elevation': content['metadata'].get('sun_elevation'),
                'target_azimuth': content['metadata'].get('target_azimuth'),
                'disaster': disaster,
                'disaster_type': content['metadata'].get('disaster_type'),
                'catalog_id': content['metadata'].get('catalog_id'),
                'original_width': content['metadata'].get('original_width'),
                'original_height': content['metadata'].get('original_height'),
                'width': content['metadata'].get('width'),
                'height': content['metadata'].get('height'),
                'id': content['metadata'].get('id')
            }
            
            # Separate pre and post DataFrame
            if "pre" in img_name.lower():
                pre_data.append(data)
            elif "post" in img_name.lower():
                post_data.append(data)
    
    hurricane_pre_df = pd.DataFrame(pre_data)
    hurricane_post_df = pd.DataFrame(post_data)
    
    return hurricane_pre_df, hurricane_post_df

In [3]:
# Setup
directory_path = "../Data"
# Create Dataframe
hurricane_pre_df, hurricane_post_df = create_dataframes(directory_path)

# 